In [1]:
'''
# if working on laptop on local docker:
nvidia-docker run -p 4242:8888 -v ~/dl_cas/:/notebooks -p 6006:6006 -it oduerr/tf_docker:gpu_r
'''

'\n# if working on laptop on local docker:\nnvidia-docker run -p 4242:8888 -v ~/dl_cas/:/notebooks -p 6006:6006 -it oduerr/tf_docker:gpu_r\n'

## Imports

In [2]:
# python module imports needed in customized functions:
import numpy as np
import tensorflow as tf
import sys

In [3]:
tf.__version__

'1.4.1'

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as imgplot

In [5]:
!pip install keras

In [6]:
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
#from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [7]:

import pandas as pd
#tf.set_random_seed(1)
#np.random.seed(1)
import sys
tf.__version__, sys.version_info



('1.4.1',
 sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0))

# Data read-in 

### Load small external MNIST data set when for working local on windows

In [8]:

# upload mnist_4000.pkl.gz which we have used in the DL course to home
# To be compatible with python3 and python2
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip

with gzip.open('./../mnist_4000.pkl.gz', 'rb') as f:
    if sys.version_info.major > 2:
        (X,y) = pickle.load(f, encoding='latin1')
    else:
        (X,y) = pickle.load(f)
PIXELS = len(X[0,0,0,:])

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

X = X.reshape([4000, 784])
#X = X/255 # is already normalized

print("small data before split X.shape", X.shape)
print("small data before  y.shape", y.shape) 

x_train = X[0:3000]
y_train = y[0:3000]
x_test = X[3000:4000]
y_test = y[3000:4000]


print("small data x_train.shape:", x_train.shape)
print("small data y_train.shape:",y_train.shape)
print("small data x_test.shape:",x_test.shape)
print("small data y_test.shape:",y_test.shape)

num_class= len(np.unique(y))
print("num_class:",num_class)



small data before split X.shape (4000, 784)
small data before  y.shape (4000,)
small data x_train.shape: (3000, 784)
small data y_train.shape: (3000,)
small data x_test.shape: (1000, 784)
small data y_test.shape: (1000,)
num_class: 10


### Or load full MNIST dataset directly from internet

In [9]:
'''
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

x_train = x_train.reshape(-1,784)
x_test = x_test.reshape(-1,784)

print("large data x_train.shape:", x_train.shape)
print("large data y_train.shape:",y_train.shape)
print("large data x_test.shape:",x_test.shape)
print("large data x_test.shape:",y_test.shape)
'''

'\nfrom keras.datasets import mnist\n\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\n# if images are not flatten (like in mnist) we need first to flatten them\n# now flatten images for fc ladder\n\nx_train = x_train.reshape(-1,784)\nx_test = x_test.reshape(-1,784)\n\nprint("large data x_train.shape:", x_train.shape)\nprint("large data y_train.shape:",y_train.shape)\nprint("large data x_test.shape:",x_test.shape)\nprint("large data x_test.shape:",y_test.shape)\n'

In [10]:
x_train[:200].shape

(200, 784)

In [11]:
np.mean(x_train[:,200])

-0.0042952602

In [12]:
np.std(x_train[:,200])

0.83271211

# Lets construct a fc NN (784->500->50->10) without noise and unsupervised task to get a benchmark for the loss and accuracy

In [13]:
### Relu 
name = 'relu'
model = Sequential()

model.add(Dense(500, batch_input_shape=(None, 784),
                kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              # optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                25050     
_________________________________________________________________
batch_normalization_2 (Batch (None, 50)                200       
_________________________________________________________________
activation_2 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
Total para

# Training

In [15]:
def convertToOneHot(vector, num_classes=None):
    result = np.zeros((len(vector), num_classes), dtype='int32')
    result[np.arange(len(vector)), vector] = 1
    return result

In [16]:
tensorboard = keras.callbacks.TensorBoard(
    log_dir='tensorboard/mnist_small/' + name + '/', 
    write_graph=True,
    histogram_freq=1
)

In [17]:
history = model.fit(x_train, 
          convertToOneHot(y_train,10), 
          nb_epoch=100, 
          batch_size=128, 
          callbacks=[tensorboard],
          validation_data=[x_test, convertToOneHot(y_test,10)], verbose=2)



/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3000 samples, validate on 1000 samples
Epoch 1/100
 - 2s - loss: 0.9742 - acc: 0.7400 - val_loss: 0.4937 - val_acc: 0.8690
Epoch 2/100
 - 1s - loss: 0.3755 - acc: 0.9307 - val_loss: 0.3794 - val_acc: 0.8970
Epoch 3/100
 - 1s - loss: 0.2321 - acc: 0.9637 - val_loss: 0.3423 - val_acc: 0.9060
Epoch 4/100
 - 2s - loss: 0.1495 - acc: 0.9823 - val_loss: 0.3395 - val_acc: 0.9060
Epoch 5/100
 - 2s - loss: 0.1043 - acc: 0.9890 - val_loss: 0.3253 - val_acc: 0.9150
Epoch 6/100
 - 1s - loss: 0.0733 - acc: 0.9947 - val_loss: 0.3099 - val_acc: 0.9210
Epoch 7/100
 - 2s - loss: 0.0505 - acc: 0.9980 - val_loss: 0.3008 - val_acc: 0.9280
Epoch 8/100
 - 1s - loss: 0.0347 - acc: 0.9997 - val_loss: 0.2860 - val_acc: 0.9240
Epoch 9/100
 - 2s - loss: 0.0260 - acc: 0.9993 - val_loss: 0.2731 - val_acc: 0.9250
Epoch 10/100
 - 2s - loss: 0.0206 - acc: 1.0000 - val_loss: 0.2610 - val_acc: 0.9280
Epoch 11/100
 - 2s - loss: 0.0173 - acc: 1.0000 - val_loss: 0.2505 - val_acc: 0.9340
Epoch 12/100
 - 2s - loss:

Epoch 96/100
 - 2s - loss: 2.7998e-04 - acc: 1.0000 - val_loss: 0.2395 - val_acc: 0.9350
Epoch 97/100
 - 2s - loss: 2.6548e-04 - acc: 1.0000 - val_loss: 0.2403 - val_acc: 0.9340
Epoch 98/100
 - 2s - loss: 2.7714e-04 - acc: 1.0000 - val_loss: 0.2407 - val_acc: 0.9340
Epoch 99/100
 - 2s - loss: 2.4113e-04 - acc: 1.0000 - val_loss: 0.2407 - val_acc: 0.9350
Epoch 100/100
 - 1s - loss: 2.5944e-04 - acc: 1.0000 - val_loss: 0.2414 - val_acc: 0.9350
